# Scrapping and Processing the DATA

### Importing the packages

In [1]:
import pandas as pd
import numpy as np
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests

import craigslist
from craigslist import CraigslistPersonals

### List of cities:

In [2]:
cities = ['seattle', 'sfbay', 'honolulu','losangeles', 'sandiego', 'newyork', 'boston', 'miami', 'austin', 'houston']

### Function that extracts and some cleaning from craigslist:

In [11]:
def extraction(site):
    
    cl = CraigslistPersonals(site = site, category='mis')
    results = cl.get_results(sort_by='newest', geotagged= True, limit = 2)
    
    title = []
    has_image = []
    url = []
    geo_tag = []
    where = []
    ids = []
    datetimes = []
    has_maps = []
    
    for result in results:
        title.append(result['name'])
        has_image.append(result['has_image'])
        url.append(result['url'])
        geo_tag.append(result['geotag'])
        where.append(result['where'])
        ids.append(result['id'])
        datetimes.append(result['datetime'])
        has_maps.append(result['has_map'])

    
    def missed_age(url):
        # initiate request
        response = requests.get(url)
        HTML = response.text

        # scrape
        age = Selector(text=HTML).xpath("//p[@class='attrgroup'][2]/span/b/text()").extract()
        return age
    
    def missed_height(url):
        # initiate request
        response = requests.get(url)
        HTML = response.text

        # scrape
        height = Selector(text=HTML).xpath("//span[@class='personals_attrbubble personals_'][2]/b/text()").extract()
        return height
    
    def missed_btype(url):
        # initiate request
        response = requests.get(url)
        HTML = response.text

        # scrape
        btype = Selector(text=HTML).xpath("//p[@class='attrgroup'][1]/span[@class='personals_attrbubble personals_'][1]/b/text()").extract()
        return btype
    
    def missed_title(url):

        # initiate request
        response = requests.get(url)
        HTML = response.text

        # scrape
        title = Selector(text=HTML).xpath("//span[@class='postingtitletext']/span[@id='titletextonly']/text()").extract()
        return title

    def missed_body(url):
        response = requests.get(url)
        HTML = response.text

        # scrape
        body = Selector(text=HTML).xpath("//section[@id='postingbody']/text()").extract()
        return body

    def missed_who4who(url):
        response = requests.get(url)
        HTML = response.text

        # scrape
        m4m = Selector(text=HTML).xpath("//h2[@class='postingtitle']/span[@class='postingtitletext']/text()").extract()
        return m4m

    
    def missed_id(url):
        response = requests.get(url)
        HTML = response.text

        # scrape
        id = Selector(text=HTML).xpath("//div[@class='postinginfos']/p[@class='postinginfo'][1]/text()").extract()
        return id
    
    # Required Lists
    title = []
    body = []
    m4m = []
    ids = []
    age = []
    height = []
    btype = []
    
    for i in url:

    # Calling the function written before

        a = missed_title(i)
        b = missed_body(i)
        c = missed_who4who(i)
        d = missed_id(i)
        e = missed_age(i)
        f = missed_height(i)
        g = missed_btype(i)
        
    # Appending the extracted lists on to the lists above
        title.append(a)
        body.append(b)
        m4m.append(c)
        ids.append(d)
        age.append(e)
        height.append(f)
        btype.append(g)
        
        
    clean = body
    
    # Cleaning my body section

    # First using .join to connect at ','
    cleaning = []
    for i in clean:
        i = ','.join(i)
        i = [i]
        cleaning.append(i)
        
    # Cleaning the body of my data
    # removing the unnessary gaps and fillers in the body section

    cleaned_body = []
    for bod in cleaning:
        for bo in bod:
            bo = bo.replace('\n','')
            bo = bo.replace('    ','')
            bo = bo.replace(',', '')
            z = bo
            cleaned_body.append(z)
        
    clean = m4m
    
# #   cLEANING IDS
#     for i in title:
#         i = i.replace('[','')
#         i = i.replace(']','')
    
    cleaning_m4m = []
    for i in clean:
        i = ','.join(i)
        i = [i]
        cleaning_m4m.append(i)
        
    cleaned_m4m = []
    
    
    for bod in cleaning_m4m:
        for bo in bod:
            if bo == '':
                cleaned_body.append(np.nan)
            else:
                bo = bo.replace(' - ', '')
                bo = bo.replace('\n','')
                bo = bo.replace('    ','')
                bo = bo.replace(',', '')
            z = bo
            cleaned_m4m.append(z)
            
    #  Creating a dataframe for finding who the connection is meant for
    df_m4m = pd.DataFrame(cleaned_m4m, columns=["Who_to"])
    
    # cleaning the ids extracted from the scrapping:
    ids_scrapy = []
    
    for i in ids:
        for g in i:
            idd = [int(s) for s in g.split() if s.isdigit()]
            ids_scrapy.append(idd)
            
#         title.append(result['name'])
#         has_image.append(result['has_image'])
#         url.append(result['url'])
#         geo_tag.append(result['geotag'])
#         where.append(result['where']) 
#         ids.append(result['id'])
#         datetimes.append(result['datetime'])
#         has_maps.append(result['has_map'])
    
    titlef = [val for sublist in title for val in sublist]
    print titlef
    agef = [val for sublist in age for val in sublist]
    btypef = [val for sublist in btype for val in sublist]
    heightf = [val for sublist in height for val in sublist]
    ids_scrapyf = [val for sublist in ids_scrapy for val in sublist]
    
    df_final = pd.DataFrame([ids_scrapyf, titlef, geo_tag, where, cleaned_m4m, agef, heightf, btypef, cleaned_body, datetimes, has_image, has_maps, url]).T
    df_final.columns = ["Ids", "Title", "Geo_Tag", "Where", "Who_to", "Age", "Height", "Body_type", "Body", "Datetime", "Has_Image", "Has_Maps", "URL"]
    df_final.insert(2, "City", site)
    return df_final

### Uploading the initial table for the city:

In [12]:
cities_first=[]
for city in cities:
    cities_first.append(extraction(city))
    

[u'Halloween on H street S.E. near Olympic', u'Sexy Indian guy in Ross on NE 20th in Bellevue with family']
[u'45 from russia', u'Met you on 28r muni bus.']
[u'Hawaii Kai Costco', u'Individuality or the "soul" is therefore']
[u'subway', u'Cool folks from Soulquarious concert we chilled during jhene aiko']
[u'Searching for Min', u'John at 24 Hr Hillcrest 09:00 Sunday']
[u'Walking your dogs Tan Overcoat', u'The Joys and Power of Celibacy']
[u'BSC Wellington Locker Room 3 pm', u'Beautiful Woman at LA Fitness Stoneham']
[u'Scandals Early Afternoon- Commando', u'Weston Cup']
[u'Met you at the Create Culture party this Saturday', u'Pink Hair Mom']


ConnectionError: HTTPSConnectionPool(host='houston.craigslist.org', port=443): Max retries exceeded with url: /mis/6010679578.html (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x11672cc90>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
cities_first

In [46]:
# all_cities = pd.concat(cities_first, axis=0)

In [48]:
all_cities = pd.read_csv('../Data/Craigs_2_4')

### Uploading and appending the data to the existing data:

In [49]:
new_cities = []
for city in cities:
    new_cities.append(extraction(city))

all_new_cities = pd.concat(new_cities, axis=0)

HTTPError: 403 Client Error: Forbidden for url: http://boston.craigslist.org/search/mis?sort=date&s=0

In [ ]:
id_mask = ~all_new_cities.Ids.isin(all_cities.Ids.values)
filtered_new = all_new_cities[id_mask]

In [ ]:
combined = pd.concat([all_cities, filtered_new], axis=0)

In [ ]:
group = combined.sort_values('City')

In [ ]:
combined_01_26 = group


In [ ]:
combined_01_26.shape

In [ ]:
combined_01_26.to_csv("Craigs_2_5", encoding= "utf-8", index =False)

### Saving the data after the append


In [50]:
df = pd.read_csv("../../Craigslist_Project/Data/Craigs_2_5")

IOError: File ../../Craigslist_Project/Jupyter Code/Craigs_2_5 does not exist

In [51]:
df.shape

(8143, 14)

In [22]:
df.Body.values[7000:7008]

array([ "Hi!  2pm on Saturday afternoon furtive glances from across the street then we smiled at each other in the crosswalk (I'm the skinny guy in black jeans long black coat).  You're super cute would love to grab coffee or a drink sometime.",
       'I saw you jogging down west cliff in the rain tonight. I would love to join you.',
       'We talked at the Casino at Gratton. You had a real interesting name you gave to me. I asked if i could add you on FaceBook you said you were in a relationship. II just wanted to be friends you were hawaiin and native? Very interesting i love native culture and used to have hawaiin friends so i thought id just add you for fun.  Unfortunatly you had to go in the quickness since your ride was waiting you told me your name it was Natowy or Nakalie? i wrote it down hella quick and cant seem to find you again.  Perhaps if you cross this you can hit me up? Please let me know if your out there id cherish the time to hang out with you you dont meet many pe

In [12]:
df.isnull().sum()

Ids            26
Title          26
City            0
Geo_Tag      1894
Where        1158
Who_to        227
Age          3405
Height       3852
Body_type    3619
Body           27
Datetime       13
Has_Image      13
Has_Maps       13
URL            13
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5030 entries, 0 to 5029
Data columns (total 14 columns):
Ids          5004 non-null float64
Title        5004 non-null object
City         5030 non-null object
Geo_Tag      3136 non-null object
Where        3872 non-null object
Who_to       4803 non-null object
Age          1625 non-null float64
Height       1178 non-null object
Body_type    1411 non-null object
Body         5003 non-null object
Datetime     5017 non-null object
Has_Image    5017 non-null object
Has_Maps     5017 non-null object
URL          5017 non-null object
dtypes: float64(2), object(12)
memory usage: 550.2+ KB


In [16]:
df.Age.unique()

array([  nan,   34.,   36.,   35.,   33.,   31.,   29.,   27.,   30.,
         40.,   23.,   58.,   46.,   21.,   28.,   65.,   52.,   22.,
         54.,  100.,   26.,   47.,   42.,   48.,   49.,   24.,   68.,
         74.,   53.,   45.,   44.,   38.,   39.,   50.,   59.,   25.,
         32.,   56.,   43.,   51.,   41.,   99.,   20.,   64.,   37.,
         55.,  101.,   75.,  123.,   18.,   60.,   62.,   57.,   92.,
         19.,   67.,   72.,   63.,   69.,   80.,   93.,   79.])